In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from main import paths, im_ext, idx_1d, LAT_DEG

In [4]:
# Path to save figures, save data and OFAM model output.
fpath, dpath, xpath = paths()

ufiles = []
vfiles = []
sfiles = []
tfiles = []
year = [1981, 1981]
# year = [2070, 2101]
for y in range(year[0], year[-1] + 1):
    for i, m in enumerate(range(1, 13)):
        ufiles.append(xpath.joinpath('ocean_u_{}_{:02d}.nc'.format(y, m)))
        vfiles.append(xpath.joinpath('ocean_v_{}_{:02d}.nc'.format(y, m)))
        sfiles.append(xpath.joinpath('ocean_salt_{}_{:02d}.nc'.format(y, m)))
        tfiles.append(xpath.joinpath('ocean_temp_{}_{:02d}.nc'.format(y, m)))
du = xr.open_mfdataset(ufiles).groupby('Time.month').mean('Time')
dv = xr.open_mfdataset(vfiles).groupby('Time.month').mean('Time')
ds = xr.open_mfdataset(sfiles).groupby('Time.month').mean('Time')
dt = xr.open_mfdataset(tfiles).groupby('Time.month').mean('Time')


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:783: FutureWarning: In xarray version 0.13 `auto_combine` will be deprecated.
  coords=coords)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:783: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset` to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in in future, please use the
new `combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  coords=coords)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:783: FutureWarning: In xarray version 0.13 `auto_combine` will be deprecated.
  coords=coords)
/g/data3/hh5/public/apps/miniconda3

In [6]:
ds
du

<xarray.Dataset>
Dimensions:         (month: 12, nv: 2, st_edges_ocean: 52, st_ocean: 51, xu_ocean: 1750, yu_ocean: 300)
Coordinates:
  * nv              (nv) float64 1.0 2.0
  * st_edges_ocean  (st_edges_ocean) float64 0.0 5.0 10.0 ... 4.056e+03 5e+03
  * st_ocean        (st_ocean) float64 2.5 7.5 12.5 ... 3.603e+03 4.509e+03
  * xu_ocean        (xu_ocean) float64 120.0 120.1 120.2 ... 294.7 294.8 294.9
  * yu_ocean        (yu_ocean) float64 -15.0 -14.9 -14.8 ... 14.7 14.8 14.9
  * month           (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    Time_bounds     (month, nv) timedelta64[ns] dask.array<shape=(12, 2), chunksize=(1, 2)>
    average_DT      (month) timedelta64[ns] dask.array<shape=(12,), chunksize=(1,)>
    u               (month, st_ocean, yu_ocean, xu_ocean) float32 dask.array<shape=(12, 51, 300, 1750), chunksize=(1, 51, 300, 1750)>

In [ ]:
ds
# 143E, 156E, 165E, 170E, 180E, 170W, 155W, 140W, 125W, 110W, 95W
jl = [143, 156, 165, 170, 180, 190, 205, 220, 235, 250, 265]

In [ ]:
lons = [180]
lats = [-0.5, 0.5]
depths = [25, 50]
ilats = [idx_1d(ds.yu_ocean.values, lats[0]), idx_1d(ds.yu_ocean.values, lats[-1])]
ilons = [idx_1d(ds.xu_ocean.values, lons[0]), idx_1d(ds.xu_ocean.values, lons[-1])]
idepths = [idx_1d(ds.st_ocean.values, depths[0]), idx_1d(ds.st_ocean.values, depths[-1])]
ilats
ilons

In [ ]:
u = np.zeros((ds.u.shape))
dk = [ds.st_edges_ocean[i+1].item() - ds.st_edges_ocean[i].item() for i in range(len(ds.st_edges_ocean) -1)]
dy = 0.1 * LAT_DEG
for i in np.arange(ilons[0], ilons[-1] + 1):
    for j in np.arange(ilats[0], ilats[-1] + 1):
        for t in range(12):
            for k in np.arange(idepths[0], idepths[1] + 1):
                # Velocity x Depth x Width.
                u[t, k, j, i] = ds.u[t, k, j, i]*dk[k]*dy

In [ ]:
u[:, idepths[0], ilats[0], ilons[0]]